In [1]:
import scipy as sp
from scipy import linalg 

In [2]:
n_snps=8000
n_ind=1000

In [3]:
#Without LD
x = sp.randn(n_snps,n_ind*2)

In [4]:
#With LD
conseq_r2=0
x = sp.zeros((n_snps,n_ind))
x[0] = sp.randn(n_ind)
rand_signs = sp.sign(sp.randn(n_snps))
for i in range(1,n_snps):
    x[i]=sp.sqrt(conseq_r2)*rand_signs[i]*x[i-1]+sp.randn(n_ind)*sp.sqrt(1-conseq_r2)
    

In [5]:
#LD and scaled genotypes
x = (x.T-sp.mean(x,axis=1)).T
x = (x.T/sp.std(x,axis=1)).T
xx = x@x.T
D = xx/sp.diag(xx)
D.shape
D

array([[ 1.        ,  0.02190675, -0.01282017, ...,  0.0522545 ,
         0.05599116,  0.01668638],
       [ 0.02190675,  1.        , -0.0241766 , ..., -0.01059969,
        -0.024495  , -0.02296755],
       [-0.01282017, -0.0241766 ,  1.        , ..., -0.00295389,
         0.05991693,  0.04358537],
       ...,
       [ 0.0522545 , -0.01059969, -0.00295389, ...,  1.        ,
         0.066974  ,  0.01383171],
       [ 0.05599116, -0.024495  ,  0.05991693, ...,  0.066974  ,
         1.        ,  0.04007443],
       [ 0.01668638, -0.02296755,  0.04358537, ...,  0.01383171,
         0.04007443,  1.        ]])

In [6]:
#LD scores
ldsc = sp.diag(D@D)
ldsc

array([9.04460043, 8.92530446, 9.26636221, ..., 8.8829335 , 9.08389668,
       9.15125699])

In [7]:
#Banded LD 
w=10
D_mask = sp.zeros((n_snps,n_snps))
for i in range(n_snps):
    min_i = max(0,i-w)
    max_i = min(n_snps,i+w)
    D_mask[min_i:max_i,i]=D[min_i:max_i,i]

D_mask

array([[ 1.        ,  0.02190675, -0.01282017, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.02190675,  1.        , -0.0241766 , ...,  0.        ,
         0.        ,  0.        ],
       [-0.01282017, -0.0241766 ,  1.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  1.        ,
         0.066974  ,  0.01383171],
       [ 0.        ,  0.        ,  0.        , ...,  0.066974  ,
         1.        ,  0.04007443],
       [ 0.        ,  0.        ,  0.        , ...,  0.01383171,
         0.04007443,  1.        ]])

In [8]:
#Banded LD scores
ldsc_mask = sp.diag(D_mask@D_mask)
ldsc_mask

array([1.005301  , 1.00633454, 1.01173694, ..., 1.01523293, 1.01109222,
       1.00972235])

In [9]:
#Simulate phenotypes
h2=0.8
n_sim=1000
true_betas = sp.randn(n_sim,n_snps)
y_g = true_betas@x
y_e = sp.randn(n_sim,n_ind)
y = sp.sqrt(h2) *((y_g.T / sp.std(y_g,axis=1)).T) + sp.sqrt(1-h2)*y_e
#Standardize phenotypes
y = ((y.T - sp.mean(y,axis=1))/sp.std(y,axis=1)).T


In [10]:
#Perform GWAS
x_train = x[:,:int(n_ind/2)]
y_train = y[:,:int(n_ind/2)]
x_test = x[:,int(n_ind/2):]
y_test = y[:,int(n_ind/2):]
betas = x_train@y_train.T/(n_ind/2)

In [11]:
#Full LDSC regression on GWAS sum stats
x2 = (n_ind/2)*(betas*betas).T
x_var = sp.vstack((sp.ones((n_snps,)),ldsc))
xx_i = linalg.inv(x_var@x_var.T)
h2s = sp.zeros(n_sim)
intercepts = sp.zeros(n_sim)
for i in range(n_sim):
    w = (xx_i)@x_var@x2[i,]
    intercepts[i] = w[0]
    h2s[i] = n_snps*w[1]/(n_ind/2)
print (sp.mean(intercepts))
print (sp.var(intercepts))
print (sp.mean(h2s))
print (sp.var(h2s))



0.5589686516423191
1.4845436795151181
0.8815057326402002
4.705385375266791


In [12]:
#Full Banded LDSC regression on GWAS sum stats
x2 = (n_ind/2)*(betas*betas).T
x_var = sp.vstack((sp.ones((n_snps,)),ldsc_mask))
xx_i = linalg.inv(x_var@x_var.T)
h2s = sp.zeros(n_sim)
intercepts_banded = sp.zeros(n_sim)
for i in range(n_sim):
    w = (xx_i)@x_var@x2[i,]
    intercepts_banded[i] = w[0]
    h2s[i] = n_snps*w[1]/(n_ind/2)
print (sp.mean(intercepts_banded))
print (sp.var(intercepts_banded))
print (sp.mean(h2s))
print (sp.var(h2s))



0.8447694817780202
7.694156326910681
3.3069460869371436
1896.6680279884347


In [13]:
c = sp.diag(betas.T @ D @ betas)
c_mask = sp.diag(betas.T @ D_mask @ betas)
c_diff = c/c_mask
i_diff = intercepts/intercepts_banded
print(c_diff[:10])
print (i_diff[:10])
sp.mean(c_diff)

[9.29527423 9.0300174  9.23903702 9.28954562 9.24472377 9.20400749
 9.29100548 9.23484617 9.10863439 9.11147051]
[ 0.58213124 -0.07743507  0.01254496  0.1162261  -1.01477731 -1.09138262
  0.47390171  0.68849864  0.57135044  1.65919645]


9.215566454866726

In [14]:
print(sp.corrcoef(c_diff,i_diff))
sp.cov(c_diff,i_diff)


[[ 1.         -0.09296229]
 [-0.09296229  1.        ]]


array([[ 0.01634353, -0.04281643],
       [-0.04281643, 12.97960572]])